In [6]:
import rasterio
from rasterio.windows import Window
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Dropout
from tensorflow.keras.models import Model
import glob
from skimage.transform import resize
#from separation_tiff import *

In [7]:
rasterio

<module 'rasterio' from '/home/jessakhi/.local/lib/python3.10/site-packages/rasterio/__init__.py'>

# 1. Split water masks in 256x256 thumbnails and filter them according to water presence

I discard thumbnails with only land or only water. 

I chose (arbitrary) to keep only thumbnails with at least 10% of water and at least 10% of land.  

# 4. All fonctions combined to split all data at once

### split masks = keep only thumbnails with at least 10% of water and at least 10% of land 

In [8]:
def split_selected_masks(image_path, prefix, output_folder, size=256): 
    selected_indexes=[]
    
    with rasterio.open(image_path) as src: 
        w, h = src.width, src.height
        
        for i in range(0, w, size): 
            for j in range(0, h, size): 
                window = Window(i, j, min(size, w - i), min(size, h - j))
                thumbnail = src.read(window = window)
                
                # keep only thumbnails with at least 10% of water and at least 10% of land 
                nb_water_pixels = sum(sum(sum(thumbnail)))
                
                if (nb_water_pixels > 65536 * (10/100)) and (nb_water_pixels < 65536 * (90/100)):  
                    output_path = f"{output_folder}/{prefix}_mask_tile_{i}_{j}.tif"
                    selected_indexes.append((i,j))
                    with rasterio.open(
                        output_path, 
                        'w', 
                        driver="GTiff", 
                        height = 256, 
                        width = 256, 
                        count = src.count, 
                        dtype = src.dtypes[0]
                    ) as dst: 
                        dst.write(thumbnail)
                        
    return selected_indexes

### split matching s2, keep 6bands

In [9]:
def scale_min_max(array, min=0, max=10000):
    bands = []
    for i in range(array.shape[2]):
        bands.append(array[:, :, i].astype(np.float32) / max)
    return np.dstack(bands)

def split_selected_s2_images(image_path, prefix, selected_indexes, output_folder, size=256): 
    # open "big" image
    with rasterio.open(image_path) as src: 
        w, h = src.width, src.height
        
        # crop thumbnail
        for (i,j) in selected_indexes: 
            window = Window(i, j, min(size, w - i), min(size, h - j))
            thumbnail = src.read(window = window)
 
            bands = []
            for k in range(thumbnail.shape[0]):
                # re scale all bands : fisrt divide by 10000
                x = (thumbnail[k, :, :].astype(np.float32) / 10000)
                # normalise between 0 and 1
                bands.append((x - x.min()) / (x.max() - x.min())) 
                
            normalized_thumbnail = np.dstack(bands)
            new_normalized_thumbnail= np.moveaxis(normalized_thumbnail, -1, 0)

            output_path = f"{output_folder}/{prefix}_s2_tile_{i}_{j}.tif"

            with rasterio.open(
                output_path, 
                'w', 
                driver="GTiff", 
                height = 256, 
                width = 256, 
                count = 6, # 6 bands
                dtype = np.float32
            ) as dst: 
                dst.write(new_normalized_thumbnail)

### split s1

In [10]:
def split_selected_s1_images(image_path, prefix, selected_indexes, output_folder, size=256): 
    # open "big" image
    with rasterio.open(image_path) as src: 
        w, h = src.width, src.height
        
        # crop thumbnail
        for (i,j) in selected_indexes: 
            window = Window(i, j, min(size, w - i), min(size, h - j))
            thumbnail = src.read(window = window)
 
            bands = []
            for k in range(thumbnail.shape[0]):
                # re scale all bands : fisrt divide by 100
                x = (thumbnail[k, :, :].astype(np.float32) / 100)
                # normalise between 0 and 1
                bands.append((x - x.min()) / (x.max() - x.min())) 
                
            normalized_thumbnail = np.dstack(bands)
            new_normalized_thumbnail= np.moveaxis(normalized_thumbnail, -1, 0)

            output_path = f"{output_folder}/{prefix}_s1_tile_{i}_{j}.tif"

            with rasterio.open(
                output_path, 
                'w', 
                driver="GTiff", 
                height = 256, 
                width = 256, 
                count = 2, 
                dtype = np.float32
            ) as dst: 
                dst.write(new_normalized_thumbnail)

In [11]:
def ensure_directory_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

def split_data(input_folder, output_folder, prefix): 
    # Ensure base output folder exists
    ensure_directory_exists(output_folder)

    # Paths for the mask files
    mask_path = input_folder + prefix + "/sentinel12_s2_"+ prefix + "_msk.tif"
    output_folder_mask = output_folder + "/masks/"
    ensure_directory_exists(output_folder_mask)  # Ensure this directory exists
    selected_indexes = split_selected_masks(mask_path, prefix, output_folder_mask)

    # Paths for the S2 images
    s2_path = input_folder + prefix + "/sentinel12_s2_"+ prefix + "_img.tif"
    output_folder_s2 = output_folder + "/s2/"
    ensure_directory_exists(output_folder_s2)  # Ensure this directory exists
    split_selected_s2_images(s2_path, prefix, selected_indexes, output_folder_s2)

    # Paths for the S1 images
    s1_path = input_folder + prefix + "/sentinel12_s1_"+ prefix + "_img.tif"
    output_folder_s1 = output_folder + "/s1/"
    ensure_directory_exists(output_folder_s1)  # Ensure this directory exists
    split_selected_s1_images(s1_path, prefix, selected_indexes, output_folder_s1)

    return len(selected_indexes)


In [7]:
prefix = '11'
output_folder = "/home/jessakhi/Documents/PAO/tuto_unet/data_folder"
input_folder = "/home/jessakhi/Documents/PAO/part1/"
ensure_directory_exists(output_folder)
split_data(input_folder, output_folder, prefix)

/home/jessakhi/.local/lib/python3.10/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(


207

In [12]:
len([e for e in os.listdir("/home/jessakhi/Documents/PAO/tuto_unet/data_folder/s1/") if '.xml' not in e])

2096

In [13]:


def load_dataset(data_dir, img_size=(256, 256)):
    X, Y = [], []
    image_pattern = '*.tif'
    is_s1 = '/s1' in data_dir  
    
    img_paths = glob.glob(os.path.join(data_dir, image_pattern))
    print(f"Looking for files in: {data_dir}")
    print(f"Found {len(img_paths)} image files.")
    
    for img_path in img_paths:
        print(f"Processing file: {img_path}")
        with rasterio.open(img_path) as src:
            
            bands = [1, 2] if is_s1 else list(range(1, 7))
            img = src.read(bands)
            img = np.moveaxis(img, 0, -1) 
            
           
            scale_factor = 100 if is_s1 else 10000
            img = (img.astype(np.float32) / scale_factor)  # Rescale
            img = (img - img.min()) / (img.max() - img.min())  # Normalize to [0, 1]
            img = resize(img, img_size + (len(bands),), preserve_range=True)
            X.append(img)
        
        
        mask_path = img_path.replace('*.tif', '*_msk.tif')
        if os.path.exists(mask_path):
            with rasterio.open(mask_path) as src:
                mask = src.read(1)
                mask = resize(mask, img_size, preserve_range=True, anti_aliasing=False).astype(np.uint8)
                Y.append(mask[..., None])  # Add channel dimension
        else:
            print(f"Mask file not found for: {img_path}")
    
    return np.array(X), np.array(Y)


In [14]:


def unet_model(input_size=(256, 256, 6)): 
    inputs = Input(input_size)
    c1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(p1)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.2)(c9)
    c9 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model



In [15]:
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model

def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    """Construit un bloc de convolution réutilisable avec deux couches convolutives, optionnellement avec BatchNorm"""
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal", padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal", padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def unet_model(input_size=(256, 256, 6), n_filters=64, dropout=0.1, batchnorm=True):
    inputs = Input(input_size)
    c1 = conv2d_block(inputs, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
    p4 = Dropout(dropout)(p4)

    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)

    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model


In [16]:
total_samples = 2096
test_size = 0.1

# Calculer le nombre d'échantillons pour la validation
validation_samples = int(total_samples * test_size)

# Le reste va pour l'entraînement
train_samples = total_samples - validation_samples

print(f"Échantillons d'entraînement: {train_samples}")
print(f"Échantillons de validation: {validation_samples}")
batch_size = 32

steps_per_epoch = train_samples // batch_size
validation_steps = validation_samples // batch_size

print(f"steps_per_epoch: {steps_per_epoch}")
print(f"validation_steps: {validation_steps}")


Échantillons d'entraînement: 1887
Échantillons de validation: 209
steps_per_epoch: 58
validation_steps: 6


In [17]:
import time
from sklearn.model_selection import train_test_split

# Charger vos données
data_dir = "/home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2"
X, Y = load_dataset(data_dir)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=42)






        



Looking for files in: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2
Found 2096 image files.
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_256_8448.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_4352_5632.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_4352_7680.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_8960_1536.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/9_s2_tile_2816_5376.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_6400_256.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_5632_5120.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/9_s2_tile_3584_1536.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_3840_4096.tif


/home/jessakhi/.local/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_7936_9728.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_3328_3328.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/15_s2_tile_3584_1024.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_8960_9472.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_3584_1280.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_6656_5120.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_1792_5632.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/6_s2_tile_256_10496.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_8192_1024.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_1024_2816.tif
Processing file: /home/jessakhi/Documents

Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/11_s2_tile_8704_8448.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_512_6656.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/9_s2_tile_4864_1024.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/11_s2_tile_2816_4608.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_6912_8704.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/6_s2_tile_7424_5376.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/15_s2_tile_7680_9728.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/9_s2_tile_5376_2560.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_4608_3584.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/11_s2_tile_4864_1280.tif
Processing file: /home/jessakhi/Documents/PA

Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/9_s2_tile_256_1792.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/11_s2_tile_2560_5120.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_2304_3328.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_8960_7424.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/9_s2_tile_5888_2304.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/9_s2_tile_8448_7936.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/15_s2_tile_6144_2560.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_6656_9472.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/11_s2_tile_5632_4608.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_8192_9728.tif
Processing file: /home/jessakhi/Documents/PA

...

Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_3072_7680.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/9_s2_tile_8192_6400.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_5632_2560.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_1536_1536.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/11_s2_tile_10752_9216.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/11_s2_tile_6144_3840.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/15_s2_tile_3328_6400.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_7680_1536.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/9_s2_tile_9472_5888.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_9984_6656.tif
Processing file: /home/jessakhi/Documents

Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_6400_8960.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/11_s2_tile_9216_8704.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_1024_3840.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_7424_5376.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/9_s2_tile_4608_768.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/11_s2_tile_6400_6400.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_7168_3840.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/6_s2_tile_9216_10496.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_4864_512.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_2304_10240.tif
Processing file: /home/jessakhi/Documents/

Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_8704_4096.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_6144_6400.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_8448_4864.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_6400_3840.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_3328_7936.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_7936_4096.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_1024_8960.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/9_s2_tile_8192_8192.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_8448_5376.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_8192_256.tif
Processing file: /home/jessakhi/Documents/

Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_7936_8448.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_7936_1792.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_6656_5376.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_4864_10240.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_3072_1536.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_8704_5376.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/6_s2_tile_2048_6656.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/13_s2_tile_3072_4352.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_9984_9216.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_4352_9472.tif
Processing file: /home/jessakhi/Document

Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_3840_2560.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_4608_4864.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_2304_512.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_7168_8704.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_10496_5632.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_8960_4608.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/9_s2_tile_4352_2048.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_2048_9728.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/6_s2_tile_8448_5888.tif
Processing file: /home/jessakhi/Documents/PAO/tuto_unet/data_folder/s2/16_s2_tile_7168_8448.tif
Processing file: /home/jessakhi/Documents/

In [ ]:
import time
model = unet_model()  
model.summary()

start_time = time.time()

results = model.fit(
    x=X_train, 
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=X_val  
    
)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Le temps d'exécution total est de {elapsed_time} secondes.")


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 6)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_38 (Conv2D)  │ (None, 256, 256,  │      3,520 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_38[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_36       │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_39 (Conv2D)  │ (None, 256, 256,  │     36,928 │ activation_36[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_39[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_37       │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 128, 128,  │          0 │ activation_37[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_16          │ (None, 128, 128,  │          0 │ max_pooling2d_8[… │
│ (Dropout)           │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_40 (Conv2D)  │ (None, 128, 128,  │     73,856 │ dropout_16[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_40[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_38       │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_41 (Conv2D)  │ (None, 128, 128,  │    147,584 │ activation_38[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_41[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_39       │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_9     │ (None, 64, 64,    │          0 │ activation_39[0]… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_17          │ (None, 64, 64,    │          0 │ max_pooling2d_9[

 Total params: 34,538,625 (131.75 MB)

 Trainable params: 34,526,849 (131.71 MB)

 Non-trainable params: 11,776 (46.00 KB)

In [33]:


def read_image_with_rasterio(path):
    with rasterio.open(path) as src:
        # Read the image as a multidimensional NumPy array
        img = src.read()
        # rasterio reads data in (bands, height, width) format,
        # you might need to transpose it depending on your use case
        img = np.moveaxis(img, 0, -1)  # Move bands to the last dimension if needed
    return img

# Example usage
image_path = "/home/jessakhi/Documents/PAO/tuto_unet/data_folder/s1/11_s1_tile_2816_256.tif"
image_array = read_image_with_rasterio(image_path)


/home/jessakhi/.local/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
